# Import

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

/home/modules/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/modules/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import numpy as np 
from os import path
import pandas as pd 
import os
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
#os.environ["CUDA_VISIBLE_DEVICES"] = '0'
#import matplotlib.pyplot as plt
import seaborn as sns

#import tensorflow_docs as tfdocs
#import tensorflow_docs.plots
#import tensorflow_docs.modeling

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import rcParams
tickfontsize=20
labelfontsize = tickfontsize

import datetime

import math

import time

In [3]:
import pickle

In [4]:
import random

In [35]:
from scipy.spatial import cKDTree
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import PCA
import random
import time
import sys

In [171]:
from sklearn.metrics import r2_score as r2

# Define variables which will be inputs

In [149]:
algo = 'xgb' #am I using XGBoost (xgb) or Neural Nets (nn)?

In [6]:
total_frac = 1 #total fraction of data set to work with

In [113]:
training_pct = .7 #how much percent of total fraction should be used for training
random_split = False #make True if the training data should be chosen randomly

In [67]:
n_remote = 2000 #the n_remote most remote points will be added to training set if random_split = False

In [8]:
del_defective_mofs = False #make True if you want to remove all MOFs which a '0' value for at least one geometric property
add_size_fp = False #make True if you want to add 20 feature columns, where each feature is the number of atoms in a linker
size_dependent = False #make True if the input ML-ready data contains fingerprint which does not normalize each PG feature by number of atoms
stacked = False #make True if the input ML-ready data contains pressure as feature
n_core = 1 #number of cores to use
ML_DATA_PATH = '/data/rgur/efrc/prep_data/all_v1/ml_data.csv'

In [9]:
start_str = 'filename'
end_str = 'valence_pa'

In [10]:
del_geometric_fp = False #make True if you want to ignore the geometric features

In [11]:
cat_col_names = ['oh_1', 'oh_2', 'oh_3', 'oh_4'] #names for interpenetration columns

In [12]:
Y_DATA_PATH = '/data/rgur/efrc/data_DONOTTOUCH/hMOF_allData_March25_2013.xlsx' #path to original hMOF data

In [160]:
default_params = {'objective':'reg:linear', 'colsample_bytree':0.3, 'learning_rate':0.1,
                'max_depth':15, 'alpha':10, 'n_estimators':10}

In [165]:
n_trees = 30 #number of weak learners. Bigger is better until 5000

In [183]:
save_pp = False #make True if you want to save the parity plot

# Write Functions

In [13]:
def load_ml_data():
    return pd.read_csv(ML_DATA_PATH)

In [14]:
def reduce_size(df):
    '''
    Remove unnecessary columns from df to decrease size
    '''
    return df.drop([col for col in df.keys() if 'Smiles' in col] + [col for col in 
                                                                             df.keys() if 'Unnamed' in col] + ['#_of_Linkers'] + ['Metal_ID'], axis=1)

In [15]:
def slim(df):
    '''
    Return data set which contains total_frac of original data set
    '''
    if total_frac != 1:
        df = df.sample(frac=total_frac, random_state=0)
    return df

In [16]:
def getPGcolNames(df):
    '''
    Return list of PG-relevant column names
    '''
    for ind, col in enumerate(df.columns):
        if start_str == col:
            start_col = ind + 1
        elif end_str == col:
            end_col = ind
    return list(df.columns[start_col:end_col])

In [17]:
def getNonPGcolNames():
    features = ['norm_valence_pa',
   'norm_atomic_rad_pa_(angstroms)',
     'norm_affinity_pa_(eV)',
       'norm_ionization_potential_pa_(eV)',
           'norm_electronegativity_pa']
    
    if not del_geometric_fp:
        features += ['norm_Dom._Pore_(ang.)',
                     'norm_Max._Pore_(ang.)',
                     'norm_Void_Fraction',
                     'norm_Surf._Area_(m2/g)',
                     'norm_Vol._Surf._Area',
                     'norm_Density']
    features += cat_col_names
    
    if stacked:
        features += ['norm_log_pressure']
        
    if add_size_fp:
        features += ['size_%s' %n for n in range(20)]
    
    return features

In [18]:
def merge_data(df):
    '''
    Merge gravimetric uptake
    '''
    if not stacked:
        y_data = pd.read_excel(Y_DATA_PATH)
        df = df.join(y_data[['Crystal ID#', 'CH4 cm3/g 35 bar']].set_index('Crystal ID#'), on='Crystal_ID#')
        for key in df.keys():
            if ' ' in key:
                new_key = key.replace(' ', '_')
                df[new_key] = df[key]
                df = df.drop(key, axis=1)

    return df

In [19]:
def norm_col(df, col_name):
    
    mean = df[col_name].mean()
    std = df[col_name].std()
    
    property_used = 'norm_' + col_name
    df[property_used] = (df[col_name] - mean) / std

    return mean, std, property_used

In [20]:
def prepare_target(df):
    if stacked:
        to_norm = 'vol_uptake'
    else:
        to_norm = 'CH4_cm3/g_35_bar'
    mean, std, property_used = norm_col(df, to_norm)
    return mean, std, property_used

In [21]:
def delMissing(df, property_used, reset=True):
    '''
    Remove rows from df with missing values in target
    '''
    df = df.iloc[df[property_used].dropna().index]
    if reset:
        df = df.reset_index().drop('index', axis=1)
    return df

In [22]:
def normPressure(df):
    '''
    Normalize pressure feature and globally save important metrics. Only use if stacked == True!
    '''
    df['log_pressure'] = np.log(df['pressure'].tolist())
    
    #global log_p_mean, log_p_std, max_p, min_p #save these variables globally
    
    log_p_mean, log_p_std, property_used = norm_col(df, 'log_pressure')

    max_p = max(df[property_used].tolist())
    min_p = min(df[property_used].tolist())
    
    return df, log_p_mean, log_p_std, max_p, min_p

In [23]:
def get_cat(s):
    '''
    Returns interpenetration from filename
    '''
    if 'cat' in s:
        return int(s.split('_')[-1][0])
    else:
        return 0

In [24]:
def addCat(df):
    '''
    Add normed one-hot encoded features for catenation (interpenetration) of MOF
    '''
    one_hot = [[0]*4 for i in range(len(df))]
    for i, f in enumerate(df['filename'].tolist()):
        one_hot[i][get_cat(f)] = 1
    oh_1 = []
    oh_2 = []
    oh_3 = []
    oh_4 = []
    for i in one_hot:
        oh_1.append(i[0])
        oh_2.append(i[1])
        oh_3.append(i[2])
        oh_4.append(i[3])
    df[cat_col_names[0]] = oh_1
    df[cat_col_names[1]] = oh_2
    df[cat_col_names[2]] = oh_3
    df[cat_col_names[3]] = oh_4

In [176]:
def prepToSplit():
    '''
    Carry out a series of tasks to prepare the data to be split into test and train sets
    '''
    ml_data = load_ml_data()
    global pg_cols
    pg_cols = getPGcolNames(ml_data)
    ml_data = reduce_size(ml_data)
    ml_data = slim(ml_data)
    global non_pg_cols
    non_pg_cols = getNonPGcolNames()
    ml_data = merge_data(ml_data)
    prepare_target(ml_data)
    global target_mean, target_std
    target_mean, target_std, property_used = prepare_target(ml_data)
    ml_data = delMissing(ml_data, property_used)

    if stacked:
        global log_p_mean, log_p_std, max_p, min_p
        ml_data, log_p_mean, log_p_std, max_p, min_p = normPressure(ml_data)
    addCat(ml_data)
    return ml_data, property_used

In [197]:
def NNtree(df, stacked, n_core):
    if stacked:
        non_p_features = [col for col in features if col != 'norm_log_pressure']
        filenames = df[non_p_features].drop_duplicates()['filename'].tolist()
        mat = df[non_p_features].drop_duplicates().to_numpy()
    else:
        non_p_features = features
        mat = df[non_p_features].to_numpy()
        filenames = df['filename'].tolist()
        
    sys.setrecursionlimit(10000)

    tree = cKDTree(mat)
    
    if n_core < 4:
        N_JOBS = 18
    else:
        N_JOBS = n_core
    
    nn_tree = tree.query(mat, k=2, n_jobs=N_JOBS)

    sys.setrecursionlimit(3000)
    
    return nn_tree, filenames

In [198]:
def trainTestSplit(df, property_used, rand=False):
    global features
    features = non_pg_cols + pg_cols
    print("Using following %s features" %len(features))
    for i in features:
        print(i)
    if rand:
        filenames = df['filename'].unique().tolist()
        random.shuffle(filenames)
        n_files = len(filenames)
        n_train = round(n_files*training_pct)
        train_fn = filenames[0:n_train]
        test_fn = filenames[n_train:]
    else:
        nn_tree = NNtree(df)
        dt = nn_tree[0] #distance tree
        nt = nn_tree[1] #neighbor tree

        to_sort = zip(nt[:, 0],dt[:, 1]) 
        srt = sorted(to_sort, key=lambda x: x[1], reverse=True)
        srt_inds = [x[0] for x in srt]
        train_inds = srt_inds[:n_remote]
        remaining = srt_inds[n_remote:]
        
        seed = 0
        random.Random(seed).shuffle(remaining)
        n_train_left = round(len(srt_inds)*training_pct - n_remote)

        train_inds += remaining[:n_train_left]
        train_fn = [df['filename'].iloc[ind] for ind in train_inds]
        
        test_inds = remaining[n_train_left:]
        test_fn = [df['filename'].iloc[ind] for ind in test_inds]   
        
    train_df = df[df['filename'].isin(train_fn)].reset_index().drop('index', axis=1)
    test_df = df[df['filename'].isin(test_fn)].reset_index().drop('index', axis=1)
    return train_df, test_df 

In [151]:
def alter_dtype(train_df, test_df):
    global train_label, test_label
    train_fp = train_df[features].to_numpy().astype('float32')
    train_label = train_df[property_used]
    test_fp = test_df[features].to_numpy().astype('float32')
    test_label = test_df[property_used]
    
    if algo == 'xgb':
        global train_d, test_d
        train_d = xgb.DMatrix(data=train_fp, label=train_label)
        test_d = xgb.DMatrix(data=test_fp, label=test_label)

In [166]:
def run_model(params=None):
    start = time.time()
    if algo == 'xgb':
        if params==None:
            params = default_params
        
        model = xgb.train(params, train_d, n_trees)
    end = time.time()
    print("Elapsed time during model training: ", end-start)
    return model

In [169]:
def get_rmse(a, b):
    '''
    Compute rmse between a and b
    '''
    return math.sqrt(np.mean(np.square(np.subtract(a, b))))

In [170]:
def unscale(property_name, test_predictions, train_predictions, test_label, train_label):
    '''
    Undo the scaling on predictions of test set, labels of test set, labels of training set
    '''
    mean = target_mean
    std = target_std
    res_test_predictions = (test_predictions * std) + mean
    res_test_label = (test_label * std) + mean
    res_train_label = (train_label * std) + mean    
    res_train_predictions = (train_predictions * std) + mean   
    return res_test_predictions, res_test_label, res_train_label, res_train_predictions

In [173]:
def parity_plot(model, save=False, fname=None):
    '''
    Make parity plot and save with name equal to fname
    '''
    if algo=='xgb':
        test_predictions = model.predict(test_d)
        train_predictions = model.predict(train_d)
    res_test_predictions, res_test_label, res_train_label, res_train_predictions = unscale(property_used, 
                                                                                           test_predictions, 
                                                                                           train_predictions, 
                                                                                           test_label, train_label)
    fig1,ax1 = plt.subplots(figsize = (8,8))


    rmse = get_rmse(res_test_label, res_test_predictions)

    tr_rmse = get_rmse(res_train_label, res_train_predictions)

    from sklearn.metrics import r2_score as r2

    r2_val = r2(y_true=res_test_label, y_pred=res_test_predictions)
    r2_tr = r2(y_true=res_train_label, y_pred=res_train_predictions)

    print("This is Test RMSE: ", rmse)
    print("This is Train RMSE: ", tr_rmse)

    ax1.scatter(res_test_label, res_test_predictions, c='r',s=10, label='Test')
    ax1.set_xlabel('True Volumetric Uptake',fontsize=labelfontsize)
    ax1.set_ylabel('Predicted Volumetric Uptake',fontsize=labelfontsize)
    max_val = max([max(res_test_label),max(res_test_predictions)])+1
    ax1.set_xlim(0, max_val)
    ax1.set_ylim(0, max_val)

    plot_x_min, plot_x_max = plt.xlim()
    plot_y_min, plot_y_max = plt.ylim()

    ax1.plot(np.linspace(plot_x_min,plot_x_max,100),np.linspace(plot_y_min,plot_y_max,100),c='k',ls='--')
    text_position_x = plot_x_min + (plot_x_max - plot_x_min) * 0.05
    text_position_y = plot_y_max - (plot_y_max - plot_y_min) * 0.25

    ax1.text(text_position_x, text_position_y, "RMSE test=" + str("%.4f" % rmse) + '\n' + 
             "RMSE train=" + str("%.4f" % tr_rmse) + '\n' +
             "R2 test=" + str("%.4f" % r2_val) + '\n' +
             "R2 train=" + str("%.4f" % r2_tr), ha='left', fontsize=16)
    fig1.tight_layout()

    if save:
        plt.savefig(fname,dpi=200)

# Scratch Cells

In [199]:
rand = False
global features
features = non_pg_cols + pg_cols
print("Using following %s features" %len(features))
for i in features:
    print(i)
if rand:
    filenames = df['filename'].unique().tolist()
    random.shuffle(filenames)
    n_files = len(filenames)
    n_train = round(n_files*training_pct)
    train_fn = filenames[0:n_train]
    test_fn = filenames[n_train:]
else:
    nn_tree, filenames = NNtree(df, stacked, n_core)
    dt = nn_tree[0] #distance tree
    nt = nn_tree[1] #neighbor tree

    to_sort = zip(nt[:, 0],dt[:, 1]) 
    srt = sorted(to_sort, key=lambda x: x[1], reverse=True)
    srt_inds = [x[0] for x in srt]
    train_inds = srt_inds[:n_remote]
    print("Top five distance: ", train_inds[0:5])
    remaining = srt_inds[n_remote:]

    seed = 0
    random.Random(seed).shuffle(remaining)
    n_train_left = round(len(srt_inds)*training_pct - n_remote)

    train_inds += remaining[:n_train_left]
    train_fn = [filenames[ind] for ind in train_inds]

    test_inds = remaining[n_train_left:]
    test_fn = [filenames[ind] for ind in test_inds]   

train_df = df[df['filename'].isin(train_fn)].reset_index().drop('index', axis=1)
test_df = df[df['filename'].isin(test_fn)].reset_index().drop('index', axis=1)

Using following 419 features
norm_valence_pa
norm_atomic_rad_pa_(angstroms)
norm_affinity_pa_(eV)
norm_ionization_potential_pa_(eV)
norm_electronegativity_pa
norm_Dom._Pore_(ang.)
norm_Max._Pore_(ang.)
norm_Void_Fraction
norm_Surf._Area_(m2/g)
norm_Vol._Surf._Area
norm_Density
oh_1
oh_2
oh_3
oh_4
Mafp_Br1_C2_C1
Mafp_Br1_C2_C2
Mafp_Br1_C2_C3
Mafp_Br1_C3_Br1
Mafp_Br1_C3_C1
Mafp_Br1_C3_C2
Mafp_Br1_C3_C3
Mafp_Br1_C3_C4
Mafp_Br1_C3_N1
Mafp_Br1_C3_N2
Mafp_Br1_C3_N3
Mafp_Br1_C3_O1
Mafp_Br1_C4_Br1
Mafp_Br1_C4_C2
Mafp_Br1_C4_C3
Mafp_Br1_C4_C4
Mafp_Br1_C4_H1
Mafp_Br1_C4_N1
Mafp_Br1_C4_N2
Mafp_Br1_C4_N3
Mafp_Br1_C4_O1
Mafp_Br1_C4_O2
Mafp_Br1_N2_C2
Mafp_Br1_N2_C3
Mafp_Br1_N2_C4
Mafp_Br1_N2_N1
Mafp_Br1_N2_N2
Mafp_Br1_N2_N3
Mafp_Br1_N3_Br1
Mafp_Br1_N3_C2
Mafp_Br1_N3_C3
Mafp_Br1_N3_H1
Mafp_Br1_N3_N2
Mafp_Br1_N3_O2
Mafp_Br1_O2_C2
Mafp_Br1_O2_C3
Mafp_Br1_O2_C4
Mafp_C1_C2_C2
Mafp_C1_C2_C3
Mafp_C1_C2_C4
Mafp_C1_C2_F1
Mafp_C1_C2_H1
Mafp_C1_C2_O1
Mafp_C1_C2_O2
Mafp_C1_C3_C2
Mafp_C1_C3_C3
Mafp_C1_C3_C4
Mafp

In [212]:
tmp = pd.DataFrame({"a": [1,2,3,4,5,20,200]})

In [213]:
tree = cKDTree(tmp)

In [215]:
nn_tree = tree.query(tmp, k=2, n_jobs=1)

In [216]:
dt = nn_tree[0] #distance tree
nt = nn_tree[1] #neighbor tree

to_sort = zip(nt[:, 0],dt[:, 1]) 
srt = sorted(to_sort, key=lambda x: x[1], reverse=True)

In [217]:
srt

[(6, 180.0), (5, 15.0), (0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0)]

In [ ]:
    
    
    if n_core < 4:
        N_JOBS = 18
    else:
        N_JOBS = n_core
    
    nn_tree = tree.query(mat, k=2, n_jobs=N_JOBS)

# Test Functions

In [177]:
ml_data, property_used = prepToSplit()

/home/modules/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
